In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow.keras as keras

In [2]:
import joblib

In [3]:
final_rmse = 5000

In [4]:
def download_data(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    return df

In [5]:
def prepare_data(df, sequence_length):
    data = df.filter(['Close'])
    df = df.dropna()
    df = df[~df.index.duplicated(keep='last')]
    df = df.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []

    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    split_index = int(len(scaled_data) * 0.8)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    
    return X_train, X_test, y_train, y_test, scaler

In [6]:
def build_lstm_model(X_train, units=50, layers=2, activation='tanh', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    for _ in range(layers - 1):
        model.add(LSTM(units, return_sequences=True))
    model.add(LSTM(units*2))
    model.add(Dense(25, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [7]:
def train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers):
    
    global final_rmse
    
    loss_history = keras.callbacks.History()
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'model_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.h5', monitor='val_loss', save_best_only=True)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping, model_checkpoint, loss_history])

    lstm_loss_history = loss_history.history['loss']
    
    lstm_predictions = model.predict(X_test)
    lstm_predictions = scaler.inverse_transform(lstm_predictions)
    
    y_test = y_test.reshape(-1,1)
    y_test = scaler.inverse_transform(y_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, lstm_predictions))
    print(f"Root Mean Squared Error (Testing Dataset): {rmse}")
    
    if rmse <= final_rmse:
        final_rmse = rmse
        joblib.dump(model, "LTC_model_lstm.pkl")
        plt.plot(y_test, label='Actual')
        plt.plot(lstm_predictions, label='Predicted')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.legend()
        plt.savefig(f'Plot_actualvspredicted/LTC_actual_vs_predicted.png')
        plt.close()
    
        plt.figure(figsize=(10, 6))
        plt.plot(loss_history.history['loss'], label='Training Loss')
        plt.plot(loss_history.history['val_loss'], label='Validation Loss')
        plt.title('Epoch Loss Curve')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(f'loss_curve/LTC_loss_curve.png')
        plt.close()
    
    return model, rmse


In [8]:
symbol = 'LTC-USD'
start_date = '2020-01-01'
end_date = '2024-01-31'

In [9]:
epochs_range = [100]
batch_sizes = [4,8]
sequence_lengths = [25,60]
units_range = [50, 100]
layers_range = [2, 3]

In [10]:
df = download_data(symbol, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [11]:
results = []

In [12]:
for epochs in epochs_range:
    for batch_size in batch_sizes:
        for sequence_length in sequence_lengths:
            for units in units_range:
                for layers in layers_range:
                    print(f"Training model with epochs={epochs}, batch_size={batch_size}, sequence_length={sequence_length}, units={units}, layers={layers}")
                    X_train, X_test, y_train, y_test, scaler = prepare_data(df, sequence_length)
                    model = build_lstm_model(X_train, units=units, layers=layers)
                    model, rmse = train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers)
                    results.append({'epochs': epochs, 'batch_size': batch_size, 'sequence_length': sequence_length, 'units': units, 'layers': layers, 'rmse': rmse})

Training model with epochs=100, batch_size=4, sequence_length=25, units=50, layers=2


Epoch 1/100

268/268 [==============================] - 17s 33ms/step - loss: 0.0076 - val_loss: 3.1342e-04
Epoch 2/100
  3/268 [..............................] - ETA: 7s - loss: 0.0039    

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 7s 25ms/step - loss: 0.0026 - val_loss: 5.0144e-04
Epoch 3/100
268/268 [==============================] - 7s 25ms/step - loss: 0.0017 - val_loss: 1.9104e-04
Epoch 4/100
268/268 [==============================] - 7s 26ms/step - loss: 0.0014 - val_loss: 2.2305e-04
Epoch 5/100
268/268 [==============================] - 6s 24ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 6/100
268/268 [==============================] - 7s 25ms/step - loss: 0.0011 - val_loss: 1.4564e-04
Epoch 7/100
268/268 [==============================] - 7s 26ms/step - loss: 0.0011 - val_loss: 7.4196e-04
Epoch 8/100
268/268 [==============================] - 7s 26ms/step - loss: 0.0011 - val_loss: 1.9088e-04
Epoch 9/100
268/268 [==============================] - 7s 28ms/step - loss: 9.7079e-04 - val_loss: 1.2579e-04
Epoch 10/100
268/268 [==============================] - 10s 36ms/step - loss: 0.0010 - val_loss: 1.6111e-04
Epoch 11/100
268/268 [==============================] - 

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 11s 41ms/step - loss: 0.0044 - val_loss: 0.0030
Epoch 3/100
268/268 [==============================] - 11s 42ms/step - loss: 0.0024 - val_loss: 2.2720e-04
Epoch 4/100
268/268 [==============================] - 10s 38ms/step - loss: 0.0019 - val_loss: 2.1134e-04
Epoch 5/100
268/268 [==============================] - 10s 38ms/step - loss: 0.0014 - val_loss: 1.8478e-04
Epoch 6/100
268/268 [==============================] - 10s 37ms/step - loss: 0.0014 - val_loss: 1.6471e-04
Epoch 7/100
268/268 [==============================] - 10s 37ms/step - loss: 0.0011 - val_loss: 6.3752e-04
Epoch 8/100
268/268 [==============================] - 10s 36ms/step - loss: 0.0011 - val_loss: 5.3976e-04
Epoch 9/100
268/268 [==============================] - 10s 38ms/step - loss: 8.5000e-04 - val_loss: 2.3830e-04
Epoch 10/100
268/268 [==============================] - 16s 59ms/step - loss: 0.0011 - val_loss: 1.4895e-04
Epoch 11/100
268/268 [==========================

...

134/134 [==============================] - 9s 70ms/step - loss: 0.0045 - val_loss: 3.2740e-04
Epoch 3/100
134/134 [==============================] - 9s 70ms/step - loss: 0.0036 - val_loss: 2.8320e-04
Epoch 4/100
134/134 [==============================] - 9s 69ms/step - loss: 0.0020 - val_loss: 5.1992e-04
Epoch 5/100
134/134 [==============================] - 9s 69ms/step - loss: 0.0018 - val_loss: 3.1547e-04
Epoch 6/100
134/134 [==============================] - 9s 69ms/step - loss: 0.0015 - val_loss: 3.4666e-04
Epoch 7/100
134/134 [==============================] - 10s 71ms/step - loss: 0.0017 - val_loss: 1.9466e-04
Epoch 8/100
134/134 [==============================] - 9s 70ms/step - loss: 0.0019 - val_loss: 4.2257e-04
Epoch 9/100
134/134 [==============================] - 10s 71ms/step - loss: 0.0013 - val_loss: 2.2537e-04
Epoch 10/100
134/134 [==============================] - 9s 71ms/step - loss: 0.0013 - val_loss: 2.2653e-04
Epoch 11/100
134/134 [==============================] -

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 10s 74ms/step - loss: 0.0028 - val_loss: 6.5386e-04
Epoch 3/100
134/134 [==============================] - 10s 72ms/step - loss: 0.0022 - val_loss: 3.8434e-04
Epoch 4/100
134/134 [==============================] - 10s 72ms/step - loss: 0.0015 - val_loss: 4.3145e-04
Epoch 5/100
134/134 [==============================] - 10s 73ms/step - loss: 0.0013 - val_loss: 1.5511e-04
Epoch 6/100
134/134 [==============================] - 10s 77ms/step - loss: 0.0011 - val_loss: 1.6765e-04
Epoch 7/100
134/134 [==============================] - 11s 80ms/step - loss: 0.0014 - val_loss: 1.4991e-04
Epoch 8/100
134/134 [==============================] - 10s 78ms/step - loss: 9.6314e-04 - val_loss: 4.1498e-04
Epoch 9/100
134/134 [==============================] - 10s 74ms/step - loss: 9.5469e-04 - val_loss: 5.2168e-04
Epoch 10/100
134/134 [==============================] - 10s 78ms/step - loss: 8.6715e-04 - val_loss: 1.2088e-04
Epoch 11/100
134/134 [==============

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 20s 147ms/step - loss: 0.0036 - val_loss: 4.1255e-04
Epoch 3/100
134/134 [==============================] - 20s 149ms/step - loss: 0.0024 - val_loss: 2.4098e-04
Epoch 4/100
134/134 [==============================] - 18s 133ms/step - loss: 0.0019 - val_loss: 2.0622e-04
Epoch 5/100
134/134 [==============================] - 18s 131ms/step - loss: 0.0015 - val_loss: 7.4789e-04
Epoch 6/100
134/134 [==============================] - 17s 130ms/step - loss: 0.0016 - val_loss: 2.6935e-04
Epoch 7/100
134/134 [==============================] - 18s 136ms/step - loss: 0.0012 - val_loss: 2.8552e-04
Epoch 8/100
134/134 [==============================] - 13s 95ms/step - loss: 0.0017 - val_loss: 4.6870e-04
Epoch 9/100
134/134 [==============================] - 16s 123ms/step - loss: 0.0010 - val_loss: 1.3735e-04
Epoch 10/100
134/134 [==============================] - 18s 131ms/step - loss: 8.7163e-04 - val_loss: 1.5698e-04
Epoch 11/100
134/134 [==============

In [13]:
for result in results:
    print(result)

{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 3.5649441456419657}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 3, 'rmse': 3.2605241961169904}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 2, 'rmse': 2.745883073945297}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 3, 'rmse': 3.1647649199138193}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 60, 'units': 50, 'layers': 2, 'rmse': 3.109352861503208}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 60, 'units': 50, 'layers': 3, 'rmse': 3.1380105579063278}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 60, 'units': 100, 'layers': 2, 'rmse': 3.48492639562897}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 60, 'units': 100, 'layers': 3, 'rmse': 2.82759218669183}
{'epochs': 100, 'batch_size': 8, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 2.737554062397694}
{'epochs': 10